In [1]:
import pandas as pd
import os
from itertools import chain
import numpy as np

In [2]:
directory = '../Data'  # Directory where the pickle files are stored
pickle_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.pickle')]

dataframes = []
for file in pickle_files:
    s = pd.read_pickle(file)
    flattened_list = list(chain.from_iterable(s))

    # Create a DataFrame
    df = pd.DataFrame(flattened_list, columns=['ID', 'DateTime'])
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)


In [3]:
counts=combined_df.groupby('ID').count().sort_values(by='ID').reset_index()
total_counts = counts['DateTime'].sum()


# Calculate the density for each 'ID'
density = counts['DateTime'] / total_counts


counts['density'] = density
grid_30x30 = np.zeros((30, 30))

# Iterate over the DataFrame and assign densities to the grid
for index, row in counts.iterrows():
    # Ensure the ID is within the valid range (0 to 899)
    if 0 <= row['ID'] < 900:
        row_idx = row['ID'] // 30  # Calculate row index
        col_idx = row['ID'] % 30   # Calculate column index
        if grid_30x30[int(row_idx), int(col_idx)]==0:
            grid_30x30[int(row_idx), int(col_idx)] = row['density']
        else:
            print("Duplicate")
            print(int(row_idx), int(col_idx))

In [5]:
import gurobipy as gp
from gurobipy import GRB
def check_gurobi_license():
    try:
        # This will initialize a Gurobi environment
        env = gp.Env()
        print("Gurobi license is valid.")
        env.dispose()  # Dispose of the environment object when done
    except gp.GurobiError as e:
        print(f"Gurobi Error: {e}")

check_gurobi_license()

Restricted license - for non-production use only - expires 2025-11-24
Gurobi license is valid.


In [ ]:
def update_density_grid(current_time,combined_df):
    # Code to update the density grid every 10 days
    subset= combined_df[combined_df.DateTime<current_time+pd.Timedelta(days=10)]
    
    counts=subset.groupby('ID').count().sort_values(by='ID').reset_index()
    total_counts = counts['DateTime'].sum()
    

    # Calculate the density for each 'ID'
    density = counts['DateTime'] / total_counts


    counts['density'] = density
    counts
    grid_30x30 = np.zeros((30, 30))

    # Iterate over the DataFrame and assign densities to the grid
    for index, row in counts.iterrows():
        # Ensure the ID is within the valid range (0 to 899)
        if 0 <= row['ID'] < 900:
            row_idx = row['ID'] // 30  # Calculate row index
            col_idx = row['ID'] % 30   # Calculate column index
            if grid_30x30[int(row_idx), int(col_idx)]==0:
                grid_30x30[int(row_idx), int(col_idx)] = row['density']
            else:
                print("Duplicate")
                print(int(row_idx), int(col_idx))
    
    return grid_30x30

def solve_p_median(grid, p):
    m = gp.Model("p_median")

    # Assuming grid is a 2D array where grid[i][j] represents the accident density at that cell

    num_rows = len(grid)
    num_cols = len(grid[0])
    num_cells = num_rows * num_cols

    # Decision Variables
    # x[i, j] = 1 if a facility is placed at cell (i, j), otherwise 0
    x = m.addVars(num_rows, num_cols, vtype=GRB.BINARY, name="x")

    # y[i, j, k, l] = 1 if cell (i, j) is assigned to facility at cell (k, l), otherwise 0
    y = m.addVars(num_rows, num_cols, num_rows, num_cols, vtype=GRB.BINARY, name="y")

    # Objective Function: Minimize the weighted sum of distances
    m.setObjective(gp.quicksum(grid[i][j] * y[i, j, k, l] * (np.abs(i-k)+ np.abs(j-l))
                               for i in range(num_rows) for j in range(num_cols)
                               for k in range(num_rows) for l in range(num_cols)), GRB.MINIMIZE)

    # Constraints

    # Constraint 1: Exactly p facilities
    m.addConstr(x.sum() == p, "facility_count")

    # Constraint 2: Each cell must be assigned to one facility
    for i in range(num_rows):
        for j in range(num_cols):
            m.addConstr(gp.quicksum(y[i, j, k, l] for k in range(num_rows) for l in range(num_cols)) == 1, f"assign_{i}_{j}")

    # Constraint 3: A cell can be assigned to a facility only if the facility is open
    for i in range(num_rows):
        for j in range(num_cols):
            for k in range(num_rows):
                for l in range(num_cols):
                    m.addConstr(y[i, j, k, l] <= x[k, l], f"open_{i}_{j}_{k}_{l}")

    # Solve the model
    m.optimize()

    # Extract and print the solution
    if m.status == GRB.OPTIMAL:
        solution_x = m.getAttr('x', x)
        solution_y = m.getAttr('x', y)

        # Print the location of facilities
        print("Facility Locations:")
        for i in range(num_rows):
            for j in range(num_cols):
                if solution_x[i, j] > 0.5:
                    print(f"Facility at cell ({i}, {j})")

        # Additional code to interpret and use the solution_y variables may be added as needed
    else:
        print("No optimal solution found")


# Main loop
current_time=combined_df.DateTime.min()
while (current_time<combined_df.DateTime.max()):
    density_grid = update_density_grid(current_time,combined_df)
    solve_p_median(density_grid, 3)
    current_time+=pd.Timedelta(days=10)


In [ ]:
for _ in range(3):
    density_grid = update_density_grid()
    solve_p_median(density_grid, p)